In [128]:
import json
with open("board.json", "r") as f:
    board = json.load(f)
print(f"Loaded {len(board)} actions.")

Loaded 5 actions.


In [129]:
def buildTree(board, getCost, costRemaining):
    return [
        {
            "move": move,
            "costRemaining": costRemaining - getCost(move),
            "nextMoves": buildTree([x for x in board if x != move], getCost, costRemaining - getCost(move)),
        }
        for move in board
        if getCost(move) <= costRemaining
    ]

In [130]:
attackerTree = buildTree(board, lambda x: x["attackerCost"], 10)
defenderTree = buildTree(board, lambda x: x["defenderCost"], 10)

In [131]:
import pygambit

In [132]:
game = pygambit.Game.new_tree()
game.title="subgame"
attacker = game.players.add("Attacker")
defender = game.players.add("Defender")

In [133]:
def labelFor(treeMove):
    return f"{treeMove['attack']['comp']},{treeMove['attack']['vuln']}"

def valueFor(treeMove):
    return treeMove["severity"]**2 * treeMove["attackerProb"]

In [134]:
def addMoves(parent, nextMoves, player, totalValue):
    if len(nextMoves) == 0:
        outcome = game.outcomes.add("value")
        outcome[attacker.number] = int(totalValue)
        outcome[defender.number] = 0
        parent.outcome = outcome
    else:
        gameMove = parent.append_move(player, len(nextMoves))
        for i, treeMove in enumerate(nextMoves):
            gameMove.actions[i].label = labelFor(treeMove["move"])
            addMoves(parent.children[i], treeMove["nextMoves"], player, totalValue + valueFor(treeMove["move"]))

In [135]:
addMoves(game.root, attackerTree, attacker,0)

C:\Users\TeamD\AppData\Local\Temp\ipykernel_21808\1094354861.py:3: UserWarning: Another outcome with an identical label exists
  outcome = game.outcomes.add("value")


In [136]:
with open("beans.efg","w") as f:
    f.write(game.write())

[<Action [0] '0,0' at infoset 'b''' for player 'b'Attacker'' in game 'b'subgame''>, <Action [1] '0,1' at infoset 'b''' for player 'b'Attacker'' in game 'b'subgame''>, <Action [2] '0,2' at infoset 'b''' for player 'b'Attacker'' in game 'b'subgame''>, <Action [3] '0,3' at infoset 'b''' for player 'b'Attacker'' in game 'b'subgame''>, <Action [4] '1,0' at infoset 'b''' for player 'b'Attacker'' in game 'b'subgame''>]